<a href="https://colab.research.google.com/github/sydneymcolumbia/CMU/blob/main/HW5statsAMZNstock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import statsmodels.api as sm

# Read the Excel file

df = pd.read_excel('AMZNdata.xlsx', engine='openpyxl')

print(df.head())


        Date     AMZN    SP500  10-Year TB
0 2020-08-01  3167.46  3351.28     0.00562
1 2020-07-01  3164.68  3271.12     0.00536
2 2020-06-01  2758.82  3100.29     0.00653
3 2020-05-01  2442.37  3044.31     0.00648
4 2020-04-01  2474.00  2912.43     0.00622


In [ ]:
df.columns = ['Date', 'AMZN', 'SP500', '10-Year TB']


# Display the first few rows of the DataFrame to check
print(df.head())

        Date     AMZN    SP500  10-Year TB
0 2020-08-01  3167.46  3351.28     0.00562
1 2020-07-01  3164.68  3271.12     0.00536
2 2020-06-01  2758.82  3100.29     0.00653
3 2020-05-01  2442.37  3044.31     0.00648
4 2020-04-01  2474.00  2912.43     0.00622


In [ ]:

# Assuming data is loaded into a DataFrame called df
df['AMZN_Return'] = df['AMZN'].pct_change()
df['SP500_Return'] = df['SP500'].pct_change()

# Convert 10-Year TB to decimal
df['10-Year TB'] = df['10-Year TB'].apply(lambda x: x.rstrip('%') if isinstance(x, str) else x).astype('float') / 100.0

#df['10-Year TB'] = df['10-Year TB'].str.rstrip('%').astype('float') / 100.0

# Calculate excess returns
df['AMZN_Excess'] = df['AMZN_Return'] - df['10-Year TB']
df['SP500_Excess'] = df['SP500_Return'] - df['10-Year TB']

# Regression
X = df['SP500_Excess'][1:]  # Exclude the first row because of NaN from pct_change()
y = df['AMZN_Excess'][1:]
X = sm.add_constant(X)  # Adds a constant term to the predictor

model = sm.OLS(y, X)
results = model.fit()

# Beta is the coefficient of SP500_Excess
beta = results.params['SP500_Excess']

# 95% Confidence Interval
conf_int = results.conf_int().loc['SP500_Excess']

# Hypothesis Testing
if 1.33 >= conf_int[0] and 1.33 <= conf_int[1]:
    print("Fail to reject the null hypothesis")
else:
    print("Reject the null hypothesis")



Fail to reject the null hypothesis


In [ ]:
#calc value of intercept (α)

alpha = results.params['const']

print(alpha)


-0.013395532143172528


In [ ]:
#Retrieve the Standard Error of the Intercept
se_alpha = results.bse['const']

print(se_alpha)



0.00619032426588856


In [ ]:
#Calculate the t-statistic
#t = \frac{\alpha - 0}{\text{se_alpha}}
t_statistic = alpha / se_alpha

print(t_statistic)


-2.1639467607517537


In [ ]:
#test hypothesis
from scipy.stats import t

# Degrees of freedom is n - 2 (where n is the number of observations)
df = len(df) - 2
critical_t_value = t.ppf(1 - 0.025, df)  # 0.025 in each tail for a 5% significance level

if abs(t_statistic) > critical_t_value:
    print("Reject the null hypothesis that α = 0.")
else:
    print("Fail to reject the null hypothesis that α = 0.")



Reject the null hypothesis that α = 0.
